In [1]:
from cvm.utils import UnitConvert as uc
from matplotlib import pyplot as plt

from pprint import pprint

from cvm.A1 import Tetrahedron, TetraOctahedron
from cvm.sample import Sample
from cvm.vibration import ClusterVibration

from cvm.utils import get_inp, parse_formula, mixed_atomic_weight, parse_input_set, UnitConvert
import numpy as np
import pandas as pd

In [14]:
inp = parse_input_set('al_mg')

In [15]:
tmp = Tetrahedron(**inp)
tmp

Tetrahedron(None,
            {'$T_\\mathrm{FD}=800$K': Sample(None,
                    {'Al4Mg2': c1: 2.448031,  c2: 2.447459,  lambda: 1.141575
                     r0: 3.099911,  x0: 0.029049
                     Gruneisen constant: 1.769390
                     Equilibrium lattice constant: 7.932290 a.u.
                     Bulk Modulus: 590.193241 Kbar
                     Debye temperature: 348.658499 K,
                     'Al5Mg1': c1: 2.452570,  c2: 2.451837,  lambda: 1.186364
                     r0: 3.050024,  x0: 0.026825
                     Gruneisen constant: 1.809219
                     Equilibrium lattice constant: 7.804638 a.u.
                     Bulk Modulus: 648.997925 Kbar
                     Debye temperature: 359.600528 K,
                     'Al6': c1: 2.489351,  c2: 2.488623,  lambda: 1.232519
                     r0: 2.999432,  x0: 0.024801
                     Gruneisen constant: 1.848428
                     Equilibrium lattice constant: 7.675180 a.u

In [16]:
tmp['$T_\mathrm{FD}=800$K']['Al5Mg1'](300)

-38506.83183151707

In [17]:
def patch(r0):
    _1st = -0.081 + (-0.00365 * (r0 - 7.85) / 0.1)
    _2nd = 0
    _3body = 0
    _4body = 0
    
    return ((_1st, _2nd), _3body, _4body)

In [18]:
tmp['$T_\mathrm{FD}=800$K'].ie(400, UnitConvert.lc2ad(7.8))

{'1st': 0.03662957553387969,
 '2nd': -0.000669334243866615,
 'triple': 0.007586212963360595,
 'tetra': 0.0023786557685525622}

In [19]:
tmp['$T_\mathrm{FD}=800$K'].ie(400)

{'1st': -0.10577843410283094,
 '2nd': -0.09935662142379442,
 'triple': 0.0252506743381673,
 'tetra': -0.007093501979397843}

In [20]:
for l, T, c, c_, e_int in tmp():
    print('1st: %s' % (e_int['1st']))
    print('2nd: %s' % (e_int['2nd']))
    print('triple: %s' % (e_int['triple']))
    print('tetra: %s' % (e_int['tetra']))
    print('T=%s' % T, 'Concentration=%s' % c, 'Counts=%s' % c_)

1st: -0.10711196417469182
2nd: -0.1003699246866745
triple: 0.025386696983332513
tetra: -0.007227003308798885
T=300.0 Concentration=6.608469631420393e-08 Counts=19
1st: -0.10660331743929419
2nd: -0.09998400489712367
triple: 0.02533515851609991
tetra: -0.007177014969784068
T=338.8888888888889 Concentration=7.562492572269388e-08 Counts=1
1st: -0.10608161874915822
2nd: -0.09958741968148388
triple: 0.02528178349530208
tetra: -0.007124522006051848
T=377.77777777777777 Concentration=9.587409375181334e-07 Counts=26
1st: -0.10554883568329387
2nd: -0.09918169100274099
triple: 0.025227111549611436
tetra: -0.007069775096169906
T=416.66666666666663 Concentration=3.996306800935293e-06 Counts=32
1st: -0.10500625226268312
2nd: -0.09876780871127266
triple: 0.025171514636895154
tetra: -0.007012938382104039
T=455.55555555555554 Concentration=1.2608313294825382e-05 Counts=39
1st: -0.10445471866842126
2nd: -0.0983464244636707
triple: 0.025115257405559532
tetra: -0.00695412063942058
T=494.44444444444446 Con

In [29]:
a = [2,5,3,9,1]
b = [1,2,3,4,5]

tmp = np.array([a, b])
tmp

array([[2, 5, 3, 9, 1],
       [1, 2, 3, 4, 5]])

In [34]:
index = np.argsort(tmp[0])
index

array([4, 0, 2, 1, 3])

In [38]:
tmp[1, index]

array([5, 1, 3, 2, 4])

In [37]:
tmp[0, index]

array([1, 2, 3, 5, 9])

* T=900.0 Concentration=0.1322374574878373 Counts=214

# load inp in old format

In [11]:
inp = get_inp('mg_vibration_TO.json')

In [12]:
pair = inp['series'][0]['normalize']['energies']
cluster = inp['series'][0]['clusters']['energies']

In [13]:
cluster = dict(lattice=inp['series'][0]['lattice'], host=inp['series'][0]['host'], **cluster)
pair = dict(lattice=inp['series'][0]['lattice'], **pair)

In [14]:
pd.DataFrame(pair).to_csv('normalizer.csv', index=False)
pd.DataFrame(cluster).to_csv('cluster.csv', index=False)

In [71]:
from ruamel.yaml import YAML
import sys

yaml = YAML()
yaml.indent(mapping=4, sequence=4, offset=2)
yaml.default_flow_style = None
yaml.width = 120

In [72]:
with open('input.yml', 'w') as f:
    yaml.dump(inp, f)